In [2]:
import wrds
import pandas as pd
import pytz
import datetime as dt
import pandas_market_calendars as mcal

In [62]:
db = wrds.Connection(wrds_username = "connorwz")

Loading library list...
Done


In [4]:
sp500 = db.raw_sql("""
                        select a.*, b.date, b.prc, b.openprc
                        from crsp.dsp500list as a,
                        crsp.dsf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2023' and b.date<='12/31/2023'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [5]:
dse = db.raw_sql("""
                        select comnam,ncusip, namedt, nameendt,permno
                        from crsp.dsenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
dse['nameendt']=dse['nameendt'].fillna(pd.to_datetime('today'))

In [6]:
# Merge with SP500 data
sp500_full_2023 = pd.merge(sp500, dse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full_2023 = sp500_full_2023.loc[(sp500_full_2023.date>=sp500_full_2023.namedt) \
                            & (sp500_full_2023.date<=sp500_full_2023.nameendt)]
sp500_full_2023.reset_index(inplace = True,drop = True)

In [7]:
sp500_full_2023 = sp500_full_2023[["permno","date","comnam","openprc","prc"]]

In [8]:
# Negative sign means bid/ask average
sp500_full_2023[sp500_full_2023.prc <0]

,permno,date,comnam,openprc,prc
23410,11786,2023-03-10,S V B FINANCIAL GROUP,NaN,-39.37000
55298,76841,2023-06-09,BIOGEN INC,NaN,-316.89999


In [9]:
# For 76841, bid/ask average represents closing price for a stock 
# with similar numerical values of adjacent days 
sp500_full_2023[sp500_full_2023.permno == 76841].loc[sp500_full_2023.date.isin(["2023-06-08","2023-06-09","2023-06-12"])]

/var/folders/bg/9xqhvbd522v9y5gqby9zsgsm0000gn/T/ipykernel_46817/4168260694.py:3: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  sp500_full_2023[sp500_full_2023.permno == 76841].loc[sp500_full_2023.date.isin(["2023-06-08","2023-06-09","2023-06-12"])]


,permno,date,comnam,openprc,prc
54680,76841,2023-06-08,BIOGEN INC,310.69000,308.88000
55298,76841,2023-06-09,BIOGEN INC,NaN,-316.89999
55326,76841,2023-06-12,BIOGEN INC,319.76001,313.41000


In [10]:
# For 11786, this seems to be something wrong
sp500_full_2023[sp500_full_2023.permno == 11786]

,permno,date,comnam,openprc,prc
322,11786,2023-01-03,S V B FINANCIAL GROUP,232.17000,225.22000
971,11786,2023-01-04,S V B FINANCIAL GROUP,230.10001,240.06000
1023,11786,2023-01-05,S V B FINANCIAL GROUP,235.71001,232.59000
1748,11786,2023-01-06,S V B FINANCIAL GROUP,237.12000,245.78999
2092,11786,2023-01-09,S V B FINANCIAL GROUP,247.96001,249.42999
2642,11786,2023-01-10,S V B FINANCIAL GROUP,246.46001,252.67999
3274,11786,2023-01-11,S V B FINANCIAL GROUP,255.31000,254.99001
3955,11786,2023-01-12,S V B FINANCIAL GROUP,259.19000,253.82001
4386,11786,2023-01-13,S V B FINANCIAL GROUP,248.06000,252.73000
4984,11786,2023-01-17,S V B FINANCIAL GROUP,254.08000,259.98999


In [11]:
# 0 means neither closing price nor bid/ask average exists 
sp500_full_2023.prc[sp500_full_2023.prc == 0]

Series([], Name: prc, dtype: float64)

In [12]:
# There are some companies without open price which caused negative prices.
sp500_full_2023[sp500_full_2023.openprc.isna()]

,permno,date,comnam,openprc,prc
23410,11786,2023-03-10,S V B FINANCIAL GROUP,NaN,-39.37000
55298,76841,2023-06-09,BIOGEN INC,NaN,-316.89999


In [13]:
# No dirty data for open price
(sp500_full_2023.openprc <= 0).sum()

0

In [14]:
# Nan values only come from open price
sp500_full_2023[sp500_full_2023.isna().sum(axis = 1) >0]

,permno,date,comnam,openprc,prc
23410,11786,2023-03-10,S V B FINANCIAL GROUP,NaN,-39.37000
55298,76841,2023-06-09,BIOGEN INC,NaN,-316.89999


In [15]:
# There is no duplicated values 
sp500_full_2023.duplicated().sum()

0

In [50]:
# get closing-open return
sp500_full_2023["prc"] = sp500_full_2023["prc"].apply(lambda x:abs(x))
sp500_full_2023["CO_ret"] = (sp500_full_2023['prc'] - sp500_full_2023['openprc'])/sp500_full_2023['openprc']

In [18]:
# Get closing-closing return
sp500_full_2023_subset = sp500_full_2023[["permno","date","prc"]]
sp500_full_2023_subset_wide = sp500_full_2023_subset.pivot(index = "date",columns = "permno", values = "prc")
sp500_full_2023_CC_ret = sp500_full_2023_subset_wide.pct_change(fill_method = None)
sp500_full_2023_CC_ret.reset_index(inplace = True)
sp500_full_2023_subset = pd.melt(sp500_full_2023_CC_ret,id_vars = "date",var_name="permno",value_name="CC_ret")
sp500_full_2023 = sp500_full_2023.merge(sp500_full_2023_subset,on = ["permno","date"])

In [21]:
sp500_full_2023.iloc[1000:1011,]

,permno,date,comnam,openprc,prc,CO_ret,CC_ret
1000,16581,2023-01-04,INVITATION HOMES INC,29.41000,29.72000,0.010541,0.015721
1001,41080,2023-01-04,CUMMINS INC,241.59000,240.46001,-0.004677,-0.002406
1002,83443,2023-01-04,BERKSHIRE HATHAWAY INC DEL,312.00000,314.54999,0.008173,0.014972
1003,24109,2023-01-04,AMERICAN ELECTRIC POWER CO INC,95.27000,95.52000,0.002624,0.006851
1004,11891,2023-01-04,M G M RESORTS INTERNATIONAL,33.80000,35.54000,0.051479,0.067909
1005,15579,2023-01-04,TEXAS INSTRUMENTS INC,165.29500,169.17000,0.023443,0.036517
1006,56573,2023-01-04,ILLINOIS TOOL WORKS INC,221.78999,220.82001,-0.004373,0.002269
1007,62308,2023-01-05,GLOBE LIFE INC,119.09000,120.19000,0.009237,0.001500
1008,77357,2023-01-05,SYNOPSYS INC,317.75000,315.04001,-0.008529,-0.018353
1009,88860,2023-01-05,ALIGN TECHNOLOGY INC,221.89000,219.82001,-0.009329,-0.021413


In [52]:
mapping_file = pd.read_csv("mapping_file_2023_sp500.csv")
# There are companies mapped to more than one entity_id
mapping_file.groupby("PERMNO").filter(lambda sub:sub.shape[0] > 1)

,PERMNO,RP_ENTITY_ID
16,10066,179A00
17,10066,2E83D7
20,10082,229150
21,10082,686DB1
129,10560,C8C45A
...,...,...
13050,90722,D553E8
13844,92010,133899
13845,92010,3E387A
14229,92685,9732A2


In [24]:
# all mapping files
[table for table in db.list_tables(library="rpna") if "mapping" in table]

['rp_company_mapping',
 'rp_entity_mapping',
 'rpa_company_mappings',
 'rpa_entity_mappings',
 'wrds_all_mapping',
 'wrds_company_mapping',
 'wrds_entity_mapping',
 'wrds_rpa_all_mappings',
 'wrds_rpa_company_mappings',
 'wrds_rpa_entity_mappings']

In [25]:
entity_date = db.raw_sql("SELECT DISTINCT rp_entity_id, range_start, range_end from rpna.rpa_entity_mappings WHERE range_start IS NOT NULL")
# Each entity_id has multiple range_start and range_end
entity_date.groupby("rp_entity_id").filter(lambda sub: sub.shape[0] > 1)['rp_entity_id'].unique()

array(['D455F9', '8B8F7A', 'F11286', ..., 'F2E039', 'DDFA62', 'MK5LS9'],
      dtype=object)

In [26]:
# an example
entity_date[entity_date["rp_entity_id"] == "8B8F7A"]

,rp_entity_id,range_start,range_end
1,8B8F7A,2006-04-14,None
116888,8B8F7A,2000-01-01,2014-10-13
131136,8B8F7A,2000-01-01,None
260089,8B8F7A,2000-01-01,2006-04-12
303183,8B8F7A,2006-04-12,2014-10-21
361503,8B8F7A,2014-10-21,None
138106,8B8F7A,2014-10-13,None


**One possible reason is that a company (entity_id) contains maultiple securities traded in different exchanges**

In [ ]:
sql_query = """
            SELECT permno_entity.permno, permno_entity.rp_entity_id, entity_date.range_start, entity_date.range_end
            FROM
            (SELECT DISTINCT a.permno, b.rp_entity_id
            FROM (SELECT * from crsp.dse WHERE ncusip IS NOT NULL) as a,
            rpna.wrds_company_names as b
            WHERE a.ncusip = SUBSTR(b.isin, 3, 8)) as permno_entity,
            (SELECT DISTINCT rp_entity_id, range_start, range_end from rpna.rpa_entity_mappings WHERE range_start IS NOT NULL) as entity_date
            WHERE permno_entity.rp_entity_id = entity_date.rp_entity_id
            """

In [76]:
mapping_file = db.raw_sql(sql_query)

In [77]:
# Now it happens that a permno may be mapped to multiple rp_entity_id and a rp_entity_id can be mapped to multiple ranges
mapping_file.head()

,permno,rp_entity_id,range_start,range_end
0,48653,00067A,2011-05-23,None
1,48653,00067A,2018-01-18,None
2,48653,00067A,2000-01-01,None
3,18306,000AD2,2024-01-23,None
4,18306,000AD2,2015-10-23,2018-05-01


# Seperating line

In [18]:
# There are some companies which are not included in RavenPack
sp500_2023_RPid[sp500_2023_RPid.isna().sum(axis = 1) !=0].groupby(["permno","comnam"]).groups.keys()

dict_keys([(12084, 'N X P SEMICONDUCTORS N V'), (12345, 'LYONDELLBASELL INDUSTRIES N V'), (13103, 'APTIV PLC'), (13586, 'PENTAIR PLC'), (14297, 'ALLEGION PLC'), (18143, 'LINDE PLC'), (18143, 'LINDE PLC NEW'), (18724, 'AMCOR PLC'), (23570, 'G E HEALTHCARE TECHNOLOGIES INC'), (23876, 'KENVUE INC'), (23942, 'FORTREA HOLDINGS INC'), (24174, 'VERALTO CORP'), (79145, 'ROYAL CARIBBEAN CRUISES LTD')])

In [19]:
# This are companies which are not included in RavenPack
dse[dse.permno.isin([12084,12345,13103,13586,14297,18143,18724,23570,23876,23942,24174,79145])]

,comnam,ncusip,namedt,nameendt,permno
7129,N X P SEMICONDUCTORS N V,N6596X10,2010-08-06,2023-12-29,12084
8008,LYONDELLBASELL INDUSTRIES N V,N5374510,2010-10-14,2014-01-07,12345
8009,LYONDELLBASELL INDUSTRIES N V,N5374510,2014-01-08,2016-12-18,12345
8010,LYONDELLBASELL INDUSTRIES N V,N5374510,2016-12-19,2020-04-06,12345
8011,LYONDELLBASELL INDUSTRIES N V,N5374510,2020-04-07,2021-03-30,12345
8012,LYONDELLBASELL INDUSTRIES N V,N5374510,2021-03-31,2022-07-18,12345
8013,LYONDELLBASELL INDUSTRIES N V,N5374510,2022-07-19,2023-09-18,12345
8014,LYONDELLBASELL INDUSTRIES N V,N5374510,2023-09-19,2023-12-29,12345
9797,DELPHI AUTOMOTIVE PLC,G2782310,2011-11-17,2016-01-11,13103
9798,DELPHI AUTOMOTIVE PLC,G2782310,2016-01-12,2017-12-04,13103


In [20]:
# There aren't duplicated companies
sp500_2023_RPid.duplicated().sum()

0

In [70]:
# Drop those companies not matched to RavenPack
sp500_2023_RPid = sp500_2023_RPid.dropna(subset = ["PERMNO","RP_ENTITY_ID"])

In [74]:
print(sp500_full_2023[sp500_full_2023.isna().sum(axis = 1) >0])
print(sp500_2023_RPid[sp500_2023_RPid.isna().sum(axis = 1) > 0])

       permno       date                           comnam  openprc        prc  \
575     23570 2023-01-04  G E HEALTHCARE TECHNOLOGIES INC    54.13   60.49000   
23244   11786 2023-03-10            S V B FINANCIAL GROUP      NaN  -39.37000   
55299   76841 2023-06-09                       BIOGEN INC      NaN -316.89999   
62794   23942 2023-07-03             FORTREA HOLDINGS INC    33.80   36.84000   
63253   23944 2023-07-05                       PHINIA INC    29.89   36.75000   
94166   24175 2023-10-02                   W K KELLOGG CO    13.80   13.35000   
94169   24174 2023-10-02                     VERALTO CORP    83.11   85.12000   

            ret  
575         NaN  
23244 -0.628725  
55299  0.025965  
62794       NaN  
63253       NaN  
94166       NaN  
94169       NaN  
       permno       date                 comnam  openprc        prc       ret  \
24120   11786 2023-03-10  S V B FINANCIAL GROUP      NaN  -39.37000 -0.628725   
57388   76841 2023-06-09             BIOGEN I

# Seperating line

In [45]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar("NYSE")
nyse_trading_2023 = nyse.valid_days(start_date="2022-12-31",end_date="2023-12-31")

In [46]:
nyse_trading_2023

DatetimeIndex(['2023-01-03 00:00:00+00:00', '2023-01-04 00:00:00+00:00',
               '2023-01-05 00:00:00+00:00', '2023-01-06 00:00:00+00:00',
               '2023-01-09 00:00:00+00:00', '2023-01-10 00:00:00+00:00',
               '2023-01-11 00:00:00+00:00', '2023-01-12 00:00:00+00:00',
               '2023-01-13 00:00:00+00:00', '2023-01-17 00:00:00+00:00',
               ...
               '2023-12-15 00:00:00+00:00', '2023-12-18 00:00:00+00:00',
               '2023-12-19 00:00:00+00:00', '2023-12-20 00:00:00+00:00',
               '2023-12-21 00:00:00+00:00', '2023-12-22 00:00:00+00:00',
               '2023-12-26 00:00:00+00:00', '2023-12-27 00:00:00+00:00',
               '2023-12-28 00:00:00+00:00', '2023-12-29 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=250, freq=None)

# Seperating line

In [27]:
sql_query = """select rpa_date_utc,timestamp_utc,rp_entity_id,headline
                from rpna.rpa_djpr_equities_2023
            """
RP_2023 = db.raw_sql(sql_query)

In [38]:
RP_2023.head()

,rpa_date_utc,rp_entity_id,headline
timestamp_utc,,,
2023-01-01 10:25:31.105,2023-01-01,00194C,BSE: Continental Controls Ltd. - Closure Of Tr...
2023-01-01 10:25:31.435,2023-01-01,00194C,BSE: Continental Controls Ltd. - Closure Of Tr...
2023-01-01 08:31:38.519,2023-01-01,01316B,"Bilibili Hosts Annual New Year's Eve Gala - ""T..."
2023-01-01 02:00:05.447,2023-01-01,0157B1,New Film Release: Cryptid Horror Movie Brings ...
2023-01-01 07:00:11.264,2023-01-01,0157B1,The Craziest Moments From the Longest Tech Boo...


In [ ]:
# Drop same headlines happening in consecutive days
RP_2023 = RP_2023.drop_duplicates("")

In [26]:
def next_workday(date):
  if date.weekday() >= 4:
    return date+dt.timedelta(days = 7-date.weekday())
  else:
    return date+dt.timedelta(days = 1)
def headline_ret_date(ET_time):
  if ET_time.weekday()>=5:
    return next_workday(ET_time).date()
  else:
    closing_time = ET_time.replace(hour = 16,minute =30)
    if ET_time < closing_time:
      return ET_time.date()
    else:
      return next_workday(ET_time).date()

In [27]:
RP_2023_test = RP_2023.copy()
RP_2023_test.head()

,timestamp_utc,rp_entity_id,headline
0,2023-01-01 10:25:31.105,00194C,BSE: Continental Controls Ltd. - Closure Of Tr...
1,2023-01-01 10:25:31.435,00194C,BSE: Continental Controls Ltd. - Closure Of Tr...
2,2023-01-01 08:31:38.519,01316B,"Bilibili Hosts Annual New Year's Eve Gala - ""T..."
3,2023-01-01 02:00:05.447,0157B1,New Film Release: Cryptid Horror Movie Brings ...
4,2023-01-01 07:00:11.264,0157B1,The Craziest Moments From the Longest Tech Boo...


In [28]:
RP_2023_test = RP_2023_test.rename(columns= {"timestamp_utc":"timestamp"})
RP_2023_test.set_index("timestamp",inplace = True)
RP_2023_test.index = pd.to_datetime(RP_2023_test.index).tz_localize("UTC").tz_convert(pytz.timezone("America/New_York"))
RP_2023_test.head()

,rp_entity_id,headline
timestamp,,
2023-01-01 05:25:31.105000-05:00,00194C,BSE: Continental Controls Ltd. - Closure Of Tr...
2023-01-01 05:25:31.435000-05:00,00194C,BSE: Continental Controls Ltd. - Closure Of Tr...
2023-01-01 03:31:38.519000-05:00,01316B,"Bilibili Hosts Annual New Year's Eve Gala - ""T..."
2022-12-31 21:00:05.447000-05:00,0157B1,New Film Release: Cryptid Horror Movie Brings ...
2023-01-01 02:00:11.264000-05:00,0157B1,The Craziest Moments From the Longest Tech Boo...


In [29]:
RP_2023_test.reset_index(inplace = True)
# Associate headlines' times to returns' times 
RP_2023_test.timestamp = RP_2023_test.timestamp.apply(lambda x:headline_ret_date(x))

In [30]:
RP_2023_test = RP_2023_test.rename(columns = {"timestamp":"date_ret"})

In [31]:
# There are some duplicated headlines after associating to returns'dates which should
# not be dropped. For example: same headlines on Saturday and Monday go to Monday together
RP_2023_test.duplicated().sum()

2541131

In [32]:
# NO NA values
RP_2023_test.isna().sum().sum()

0

In [33]:
RP_2023_test.date_ret = pd.to_datetime(RP_2023_test.date_ret)
sp500_2023_ret_headline = sp500_2023_RPid.merge(RP_2023_test, how = "inner", left_on = ["date","RP_ENTITY_ID"],right_on = ["date_ret","rp_entity_id"])

In [34]:
# There are duplicates because of duplicates in headline data frame after associated with returns'
# dates which should be kept
sp500_2023_ret_headline.duplicated().sum()

378922

In [35]:
# sp500_2023_ret_headline = sp500_2023_ret_headline.drop_duplicates()
sp500_2023_ret_headline = sp500_2023_ret_headline[["date","permno","ret","headline"]]
sp500_2023_ret_headline.head()

,date,permno,ret,headline
0,2023-01-03,22592,0.021264,The Worldwide Industrial Food and Beverages Fi...
1,2023-01-03,22592,0.021264,Europe OTC Braces & Supports Market Report 202...
2,2023-01-03,22592,0.021264,$16.6 Billion Worldwide Ceramic Matrix Composi...
3,2023-01-03,22592,0.021264,MIMEDX Appoints Ricci S. Whitlow as Chief Oper...
4,2023-01-03,22592,0.021264,3M Tries to Contain Legal Battles Over 'Foreve...
